<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/5_21_alpha_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!/usr/bin/env python3
"""
Google Colab用 - SLSQP と trust-constr の最適化アルゴリズム比較
(K,M)=(4,4) および (K,M)=(4,2) で実行

このスクリプトは必要なすべてのファイルをColabにアップロードすることを前提としています。
以下のファイルが必要です：
1. run_optimizer_comparison_colab.py (このファイル)
2. config.py
3. utilities.py
4. analysis_flow.py
"""

import numpy as np
import pandas as pd
import os
import logging
from datetime import datetime
from google.colab import files
import sys

# プロジェクト内のインポート
import config
import utilities
from analysis_flow import run_comparison_analysis_sweep

# ロギングの設定
log_filename = f'optimizer_comparison_K4M_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    filename=log_filename,
    filemode='w'
)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)
logger = logging.getLogger(__name__)

def run_optimizer_comparison_KM(k_assets, m_models, alpha_grid=None, r_base=None, sigma_base=None,
                               corr_in=0.75, corr_out=0.50, beta_corr=1.0):
    """trust-constrとSLSQP最適化アルゴリズムの比較テスト

    Parameters:
    -----------
    k_assets : int
        資産数
    m_models : int
        モデル数
    alpha_grid : array_like, optional
        テストするα値の配列
    r_base : array_like, optional
        基準期待リターンの配列
    sigma_base : array_like, optional
        基準ボラティリティの配列
    corr_in : float, optional
        ブロック内相関係数
    corr_out : float, optional
        ブロック間相関係数
    beta_corr : float, optional
        相関行列混合パラメータ
    """
    logger.info(f"Starting optimizer comparison test for K={k_assets}, M={m_models}")

    # 出力ディレクトリの作成
    output_dir = f"optimizer_test_K{k_assets}M{m_models}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(output_dir, exist_ok=True)
    logger.info(f"Output directory: {output_dir}")

    # 基本パラメータの設定
    if r_base is None:
        r_base = np.array([0.02, 0.03, 0.04, 0.05][:k_assets])

    if sigma_base is None:
        sigma_base = np.array([0.20, 0.25, 0.30, 0.35][:k_assets])

    # Delta行列
    if m_models == 2:
      # 左端（列0）と右端（列3）を選択
      Delta_A = np.array([[+1, 1, -1, -1],
                         [+1, 1, -1, -1],
                         [+2, -1, +2, -2],
                         [+2, -2, +2, -2]], float)[:k_assets, [0, 3]]
    elif m_models == 4:
      # 全列を使用
      Delta_A = np.array([[+1, 1, -1, -1],
                         [+1, 1, -1, -1],
                         [+2, -1, +2, -2],
                         [+2, -2, +2, -2]], float)[:k_assets, :]
    else:
      # その他の場合は最初のm_models列を使用
      Delta_A = np.array([[+1, 1, -1, -1],
                         [+1, 1, -1, -1],
                         [+2, -1, +2, -2],
                         [+2, -2, +2, -2]], float)[:k_assets, :m_models]
    Delta_r = Delta_A/1000
    Delta_sigma = Delta_A/100

    # 相関行列
    def block_corr(r_in: float, r_out: float, size: int) -> np.ndarray:
        C = np.full((size, size), r_out, dtype=float)
        np.fill_diagonal(C, 1.0)

        mid = size // 2
        for i in range(mid):
            for j in range(mid):
                if i != j:
                    C[i, j] = r_in
        for i in range(mid, size):
            for j in range(mid, size):
                if i != j:
                    C[i, j] = r_in
        return C

    C_base = block_corr(corr_in, corr_out, k_assets)

    # tilde_C行列
    if m_models == 2:
        # M=2の場合の特別なtilde_C
        tilde_C = [
            block_corr(0.85, 0.60, k_assets),
            block_corr(0.65, 0.40, k_assets)
        ]
    else:
        # M=4の場合（またはその他）
        tilde_C = [
            block_corr(0.85, 0.40, k_assets),
            block_corr(0.65, 0.40, k_assets),
            block_corr(0.65, 0.60, k_assets),
            block_corr(0.85, 0.60, k_assets)
        ][:m_models]  # m_models分だけ取得

    # V_baseの作成
    sigma_diag = np.diag(sigma_base)
    V_base = sigma_diag @ C_base @ sigma_diag

    # テストするalpha値
    if alpha_grid is None:
        alpha_grid = np.linspace(0, 0.1, 6)  # 少ないグリッドで実行時間を短縮

    # ==========================================================================
    # trust-constrとSLSQPの比較テスト
    # ==========================================================================

    # === オリジナルのouter_optimizer_generalをバックアップ ===
    original_outer_optimizer = utilities.outer_optimizer_general

    try:
        # === trust-constr を使ったテスト ===
        logger.info(f"K={k_assets}, M={m_models}: Running test with trust-constr optimizer")

        # trust-constr の結果を保存
        csv_path_trust_constr = os.path.join(output_dir, f"trust_constr_K{k_assets}M{m_models}_results.csv")

        # run_comparison_analysis_sweep呼び出し
        df_trust_constr = run_comparison_analysis_sweep(
            alpha_grid=alpha_grid,
            k_assets=k_assets,
            m_models=m_models,
            mu_tilde=config.MU_TILDE,
            r_base=r_base,
            sigma_base=sigma_base,
            C_base=C_base,
            V_base=V_base,
            Delta_r_m=Delta_r,
            Delta_sigma_m=Delta_sigma,
            tilde_C_list=tilde_C,
            beta_corr=beta_corr,
            csv_path=csv_path_trust_constr,
            log_every=1
        )

        logger.info(f"K={k_assets}, M={m_models}: trust-constr results saved to {csv_path_trust_constr}")

        # === SLSQPを使ったテスト ===
        logger.info(f"K={k_assets}, M={m_models}: Running test with SLSQP optimizer")

        # SLSQP用のカスタムoptimizer関数を作成
        def slsqp_optimizer(R, Sigma, mu_tilde, **kwargs):
            # outer_opt_algが既にある場合は削除
            kwargs_copy = kwargs.copy()
            if 'outer_opt_alg' in kwargs_copy:
                del kwargs_copy['outer_opt_alg']

            return original_outer_optimizer(
                R, Sigma, mu_tilde,
                outer_opt_alg='SLSQP',
                **kwargs_copy
            )

        # オリジナルのoptimizer関数を置き換え
        utilities.outer_optimizer_general = slsqp_optimizer

        # SLSQP の結果を保存
        csv_path_slsqp = os.path.join(output_dir, f"slsqp_K{k_assets}M{m_models}_results.csv")

        # run_comparison_analysis_sweep呼び出し
        df_slsqp = run_comparison_analysis_sweep(
            alpha_grid=alpha_grid,
            k_assets=k_assets,
            m_models=m_models,
            mu_tilde=config.MU_TILDE,
            r_base=r_base,
            sigma_base=sigma_base,
            C_base=C_base,
            V_base=V_base,
            Delta_r_m=Delta_r,
            Delta_sigma_m=Delta_sigma,
            tilde_C_list=tilde_C,
            beta_corr=beta_corr,
            csv_path=csv_path_slsqp,
            log_every=1
        )

        logger.info(f"K={k_assets}, M={m_models}: SLSQP results saved to {csv_path_slsqp}")

        # === 結果の比較 ===
        # CSVからデータを読み込み（確実にインデックスを揃えるため）
        df_trust_constr = pd.read_csv(csv_path_trust_constr)
        df_slsqp = pd.read_csv(csv_path_slsqp)

        # scenarioとalphaでグループ化
        df_trust_constr['optimizer'] = 'trust-constr'
        df_slsqp['optimizer'] = 'SLSQP'

        # 結合
        df_combined = pd.concat([df_trust_constr, df_slsqp])

        # 比較用CSVとして保存
        csv_path_comparison = os.path.join(output_dir, f"optimizer_comparison_K{k_assets}M{m_models}.csv")
        df_combined.to_csv(csv_path_comparison, index=False)
        logger.info(f"K={k_assets}, M={m_models}: Combined comparison results saved to {csv_path_comparison}")

        # サマリー統計の計算
        summary_data = []

        # 各シナリオと各αについて、optimizerごとの比較を行う
        for scenario in df_combined['scenario'].unique():
            for alpha in alpha_grid:
                # そのシナリオとαの行を取得
                rows = df_combined[(df_combined['scenario'] == scenario) &
                                   (df_combined['alpha'] == alpha)]

                # trust-constrとSLSQPの行を取得
                trust_row = rows[rows['optimizer'] == 'trust-constr']
                slsqp_row = rows[rows['optimizer'] == 'SLSQP']

                if not trust_row.empty and not slsqp_row.empty:
                    # Sharpe比の比較
                    trust_sr = trust_row['robust_sharpe_ratio'].values[0]
                    slsqp_sr = slsqp_row['robust_sharpe_ratio'].values[0]
                    sr_diff = slsqp_sr - trust_sr

                    # 分散の比較
                    trust_var = trust_row['robust_variance'].values[0]
                    slsqp_var = slsqp_row['robust_variance'].values[0]
                    var_diff = slsqp_var - trust_var

                    # 繰り返し数の比較
                    trust_iters = trust_row['iters'].values[0]
                    slsqp_iters = slsqp_row['iters'].values[0]
                    iters_diff = slsqp_iters - trust_iters

                    summary_data.append({
                        'scenario': scenario,
                        'alpha': alpha,
                        'trust_constr_SR': trust_sr,
                        'SLSQP_SR': slsqp_sr,
                        'SR_diff': sr_diff,
                        'trust_constr_var': trust_var,
                        'SLSQP_var': slsqp_var,
                        'var_diff': var_diff,
                        'trust_constr_iters': trust_iters,
                        'SLSQP_iters': slsqp_iters,
                        'iters_diff': iters_diff
                    })

        # サマリーデータフレームを保存
        df_summary = pd.DataFrame(summary_data)
        csv_path_summary = os.path.join(output_dir, f"summary_K{k_assets}M{m_models}.csv")
        df_summary.to_csv(csv_path_summary, index=False)
        logger.info(f"K={k_assets}, M={m_models}: Summary statistics saved to {csv_path_summary}")

        # 差分が大きいケースをログに出力
        large_diff = df_summary[abs(df_summary['SR_diff']) > 0.001]
        if not large_diff.empty:
            logger.info(f"K={k_assets}, M={m_models}: Found {len(large_diff)} cases with large SR differences:")
            for _, row in large_diff.iterrows():
                logger.info(f"  Scenario: {row['scenario']}, Alpha: {row['alpha']}, "
                          f"SR diff: {row['SR_diff']:.6f}, "
                          f"trust-constr: {row['trust_constr_SR']:.6f}, "
                          f"SLSQP: {row['SLSQP_SR']:.6f}")

        # イテレーション数の差が大きいケースをログに出力
        iter_diff = df_summary[abs(df_summary['iters_diff']) > 5]
        if not iter_diff.empty:
            logger.info(f"K={k_assets}, M={m_models}: Found {len(iter_diff)} cases with large iteration differences:")
            for _, row in iter_diff.iterrows():
                logger.info(f"  Scenario: {row['scenario']}, Alpha: {row['alpha']}, "
                          f"Iters diff: {row['iters_diff']}, "
                          f"trust-constr: {row['trust_constr_iters']}, "
                          f"SLSQP: {row['SLSQP_iters']}")

        # Colab環境の場合、結果ファイルをダウンロード
        try:
            if 'google.colab' in sys.modules:
                logger.info(f"Downloading result files for K={k_assets}, M={m_models}")
                files.download(csv_path_trust_constr)
                files.download(csv_path_slsqp)
                files.download(csv_path_comparison)
                files.download(csv_path_summary)
        except Exception as e:
            logger.warning(f"Failed to download files: {e}")

        return output_dir

    finally:
        # オリジナルのoptimizer関数を復元
        utilities.outer_optimizer_general = original_outer_optimizer

def main(custom_params=None):
    """K=4, M=4とK=4, M=2の両方の設定で実行

    Parameters:
    -----------
    custom_params : dict, optional
        カスタムパラメータ（alpha_grid, r_base, sigma_base等）
    """
    # デフォルトパラメータ
    params = {
        'alpha_grid': np.linspace(0.3, 0.4, 11),
        'r_base_K4': np.array([0.02, 0.03, 0.04, 0.05]),
        'sigma_base_K4': np.array([0.20, 0.25, 0.30, 0.35]),
        'corr_in': 0.75,
        'corr_out': 0.50,
        'beta_corr': 1.0
    }

    # カスタムパラメータがあれば上書き
    if custom_params:
        params.update(custom_params)

    logger.info("="*80)
    logger.info("Starting optimizer comparison with parameters:")
    for key, value in params.items():
        if key.startswith('alpha_grid'):
            logger.info(f"  {key}: {list(value)}")
        elif key.startswith(('r_base', 'sigma_base')):
            logger.info(f"  {key}: {value.tolist()}")
        else:
            logger.info(f"  {key}: {value}")
    logger.info("="*80)

    # K=4, M=4の設定で実行
    logger.info("="*80)
    logger.info("Starting comparison for K=4, M=4")
    logger.info("="*80)
    output_dir_K4M4 = run_optimizer_comparison_KM(
        k_assets=4,
        m_models=4,
        alpha_grid=params['alpha_grid'],
        r_base=params['r_base_K4'],
        sigma_base=params['sigma_base_K4'],
        corr_in=params['corr_in'],
        corr_out=params['corr_out'],
        beta_corr=params['beta_corr']
    )

    # K=4, M=2の設定で実行
    logger.info("="*80)
    logger.info("Starting comparison for K=4, M=2")
    logger.info("="*80)
    output_dir_K4M2 = run_optimizer_comparison_KM(
        k_assets=4,
        m_models=2,
        alpha_grid=params['alpha_grid'],
        r_base=params['r_base_K4'],
        sigma_base=params['sigma_base_K4'],
        corr_in=params['corr_in'],
        corr_out=params['corr_out'],
        beta_corr=params['beta_corr']
    )

    logger.info("="*80)
    logger.info("Comparison completed")
    logger.info(f"K=4, M=4 results: {output_dir_K4M4}")
    logger.info(f"K=4, M=2 results: {output_dir_K4M2}")
    logger.info("="*80)

    # ログファイルをダウンロード
    try:
        if 'google.colab' in sys.modules:
            files.download(log_filename)
    except Exception as e:
        logger.warning(f"Failed to download log file: {e}")

# カスタム実行例
if __name__ == "__main__":
    # 通常の実行
    #main()

    # カスタムパラメータを指定して実行する例
    custom_params = {
        'alpha_grid': np.concatenate([
            np.linspace(0, 0.01, 3),    # 小さい値領域
            np.linspace(0.02, 0.1, 5),  # 中間領域
            np.linspace(0.2, 0.5, 3)     # 大きい値領域
        ]),
        'r_base_K4': np.array([0.03, 0.04, 0.06, 0.08]),  # リターンを大きく
        'sigma_base_K4': np.array([0.15, 0.18, 0.22, 0.30]),  # ボラティリティを調整
        'corr_in': 0.80,  # ブロック内相関を強く
        'corr_out': 0.40,  # ブロック間相関を弱く
    }

    # コメントアウトを解除して実行
    # main(custom_params)

    # デフォルトパラメータで実行
    main()

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2025-05-22 00:40:46,029 - __main__ - WARNING - Failed to download log file: Cannot find file: optimizer_comparison_K4M_20250522_004031.log
2025-05-22 00:40:46,029 - __main__ - WARNING - Failed to download log file: Cannot find file: optimizer_comparison_K4M_20250522_004031.log


In [ ]:
tilde_C = [block_corr(0.85,0.60), block_corr(0.65,0.40)]